In [ ]:

import loader
path_HE = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/Data_set_BCI_challange/train/HE_imgs/HE'
path_IHC = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/Data_set_BCI_challange/train/IHC_imgs/IHC'

In [ ]:
from torch.utils.data import DataLoader
train_data = loader.stain_transfer_dataset( epoch = 1,
                                                        num_epochs = 1,
                                                        HE_img_dir = path_HE,
                                                        IHC_img_dir = path_IHC,
                                                        img_size=  [64,64],
                                           )
train_data_loader = DataLoader(train_data, batch_size=1, shuffle=False) 

In [ ]:
HE_img, IHC_img = next(iter(train_data_loader))
print (HE_img.shape)
print (IHC_img.shape)

In [ ]:
import torch 
from torch import nn
import math

In [ ]:
class PatchEmbedding(nn.Module):
####### patch embedding ###########
#   - in_channels are the input channels of the images for rgb = 3
#   - patch_size is the sizes of the patches for the embedding 
#   - img_size is the size of the input image (has to be quadratic)
#   - embedding_dim are the dimentions for the embedding with embedding_dim=0 it will be automaticly calculated 
    def __init__(self, in_channels, patch_size, embedding_dim):
        super().__init__()
        self.in_channels = in_channels
        self.patch_size = patch_size
        self.embedding_dim = embedding_dim
# 1) the conv layer with kernel_size = stride writes out patches of the kernels the embedding_dim should be choosen as (patch_size**2)* in_channel so that no information is lost          
        self.patcher = nn.Conv2d(in_channels=in_channels,
                                 out_channels=self.embedding_dim,
                                 kernel_size=patch_size,
                                 stride=patch_size,
                                 padding=0)

# 2) flatten the feature map into 1D
        self.flatten = nn.Flatten(start_dim=2,
                                  end_dim=3)


    def forward(self, x):
      
        
        x_patched = self.patcher(x)
        x_flattened = self.flatten(x_patched) 
# 3) permute the output tensor so that it has the form [batch_size, embedding_dim, num_patches]      
        return x_flattened
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def getPositionEmbedding(embedding_dim, num_patches, n=10000):
    # the n variable is scalling the values in the positional embedding in the attention is all you need paper n=10000 was choosen 
    p_embedding = torch.zeros((embedding_dim, num_patches))
    p_embedding = p_embedding.cuda()
    for k in range(embedding_dim):
        for i in torch.arange(int(num_patches/2)):
            denominator = np.power(n, 2*i/num_patches)
            p_embedding[k, 2*i] = np.sin(k/denominator)
            p_embedding[k, 2*i+1] = np.cos(k/denominator)
    
    return torch.unsqueeze(p_embedding, dim=0)

x = getPositionEncoding(embedding_dim=764, num_patches=16, n=10)


print(x.get_device())

In [ ]:
patches = PatchEmbedding(in_channels=3,patch_size=16, embedding_dim=764)
patches.cuda()
y = patches(HE_img)

print(y.shape)

In [ ]:
# d_model = embedding_dim
# nhead = attention heads 
# https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html#torch.nn.TransformerEncoderLayer


transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=768,
                                                       nhead=12,
                                                       dim_feedforward=2048,
                                                       dropout=0.1,
                                                       activation="gelu",
                                                       batch_first=True,
                                                       norm_first=True)

transformer_decoder_layer = nn.TransformerDecoderLayer(d_model=768,
                                                       nhead=12,
                                                       dim_feedforward=2048,
                                                       dropout=0.1,
                                                       activation="gelu",
                                                       batch_first=True,
                                                       norm_first=True)

In [ ]:
transformer_encoder = nn.TransformerEncoder(
    encoder_layer=transformer_encoder_layer,
    num_layers=12)

transformer_decoder = nn.TransformerEncoder(
    encoder_layer=transformer_decoder_layer,
    num_layers=12)

In [ ]:
class Generator(nn.Module):
###### GENERATOR CLASS #############
# 1) set up the embedding of input img :
#        - img_size is the size of the input inmage of the generator 
#        - embedding_dim : the dimentions used also in the transformer
#        - patch_size : sizes of patches cut by the embedding [img_size % patch_size != 0]
#        - create positional embedding
#
# 2) set up the the transformer encoder layer class :
#        ---> https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html#torch.nn.TransformerEncoderLayer
#        - the d_model = embedding_dim so that the dimentions of the embedded image and the transformer network match
#        - nhead sets the number of heads for self attention in a transformer block 
#
# 3) set up encoder class:
#        --->  https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoder.html#torch.nn.TransformerEncoder
#        - use the encoder-layer set up in 2) 
#        - num_layers defines the number of encoder-layers in the encoder

    def __init__(self,img_size,embedding_dim, patch_size, in_channels, dropout_embedding, nhead,num_layers):
        super(Generator, self).__init__()
        
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.dropout_embedding = dropout_embedding
        self.nhead = nhead
        self.num_layers = num_layers
        self.embedding_dim = embedding_dim


        #### testting the compatebility for img_size and patch_site 
        if img_size % patch_size == 0:
            self.patch_size = patch_size
        else : 
            print('img_size / patch_size has to have no rest')

        # number of patches in image for given patchsize 
        num_patches = (img_size * img_size) // patch_size**2 
        # number of valiables in input image ( num_channels* img_height* img_width)
        num_values = in_channels * img_size**2

        if num_values % num_patches == 0:
            self.embedding_dim =  int(num_values/num_patches)
        else:
            print('num_values / patch_num has to have no rest')

        # create patches from the imput image the output by the PatchEmbedding is : [batch_size, num_patches, embedding_dim ]
        # where as the embedding_dim is choosen as patch_size**2 * in_channels 
        self.patch_embedding = PatchEmbedding(in_channels=self.in_channels,patch_size=self.patch_size, embedding_dim=self.embedding_dim)
    
        self.num_patches = (img_size * img_size) // patch_size**2 
        # positional embedding is added to keep the informatiuon of the order of the patches 
        self.positional_embedding = nn.Parameter(torch.randn(1, self.num_patches, self.embedding_dim))

        self.embedding_dropout = nn.Dropout(p=self.dropout_embedding)

        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model= self.embedding_dim,
                                                               nhead=self.nhead ,
                                                               dim_feedforward=2048,
                                                               dropout=0.1,
                                                               activation="gelu",
                                                               batch_first=True,
                                                               norm_first=True)
        
        self.transformer_encoder = nn.TransformerEncoder(
                                                    encoder_layer=self.transformer_encoder_layer,
                                                    num_layers=self.num_layers)
        
        self.upsample = nn.PixelShuffle(self.num_patches)
        
        #self.linear = nn.Sequential(nn.Conv2d(self.embedding_dim, 3, 1, 1, 0))

    def forward(self, x):
        batch_size = x.shape[0]

        # Create the patch embedding
        x = self.patch_embedding(x)

        x = self.positional_embedding + x

        x = self.embedding_dropout(x)

        x = self.transformer_encoder(x)

        x = x.permute(0, 2, 1)

        x = x.view(1, self.embedding_dim, int(math.sqrt(self.num_patches)), int(math.sqrt(self.num_patches)))

        x = self.upsample(x)

        
        return x
    



        

        

In [ ]:
from torchinfo import summary 

vit = Generator(img_size= 64,
                embedding_dim=0,
                patch_size=16,
                in_channels=3,
                dropout_embedding=0.1,
                nhead= 4,
                num_layers=6
                
                )
vit.cuda()
fake_IHC = vit(HE_img)
print(fake_IHC.shape)

In [ ]:
transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=768,
                                                       nhead=12,
                                                       dim_feedforward=2048,
                                                       dropout=0.1,
                                                       activation="gelu",
                                                       batch_first=True,
                                                       norm_first=True)
transformer_encoder_layer.cuda()

In [ ]:

upsample = nn.PixelShuffle(16)
x = fake_IHC.permute(0,2,1).view(1,768,4,4)
y = upsample(x)
print(y.shape)

In [ ]:
x = torch.zeros(1, 16, 768)

x = x.permute(0, 2, 1)
x = x.view(-1,3,64,64)
print(x.shape)
conv_layer = nn.Conv2d(in_channels =768,
                       out_channels= 3,
                       kernel_size= 1,
                       stride = 1,
                       padding= 0)

out  = conv_layer (x)
print (out.shape)

In [ ]:
a = {}
a['num'] = []
for n in range (10):
    a['num'].append(n)


# open file for writing
f = open("dict.txt","w")

# write file
f.write( str(a) )

# close file
f.close()

In [12]:
import gc
import torch 
import loader 
import conv_models
from torch.utils.data import DataLoader
import utils
from torchmetrics import StructuralSimilarityIndexMeasure
from torchmetrics import PeakSignalNoiseRatio
import numpy as np


gen_G_path = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/torch_stain_transfer/experiment-results/Test_01/Generator_G_weights.pth'
params = utils.get_config_from_yaml('C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/torch_stain_transfer/code/config.yaml')
model = conv_models.GeneratorResNet(3, num_residual_blocks=6)

model.load_state_dict(torch.load(gen_G_path))
model =  model.cuda()

model.eval()

# set up result vector 
result = {}
result['epoch'] = []
result['ssim_mean'] = []
result['ssim_std'] = []
result['psnr_mean'] = []
result['psnr_std'] = []

# set up test data dirs 
test_path = params['test_dir']
HE_img_dir = "{}{}".format(test_path,'/HE_imgs/HE')
IHC_img_dir = "{}{}".format(test_path,'/IHC_imgs/IHC')


    

test_data = loader.stain_transfer_dataset(  epoch = 1,
                                                num_epochs = params['num_epochs'],
                                                HE_img_dir = HE_img_dir,
                                                IHC_img_dir = IHC_img_dir,
                                                img_size= params['img_size'],
                                                )
    
test_data_loader = DataLoader(test_data, batch_size=1, shuffle=False) 

ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
ssim = ssim.cuda()
ssim_scores = []

psnr = PeakSignalNoiseRatio()
psnr = psnr.cuda()
psnr_scores = []

for i in range(6):
    real_HE, real_IHC = next(iter(test_data_loader))
        
    fake_IHC = model(real_HE)
    fake_IHC = fake_IHC +1

    # step 2: convert it to [0 ,1]
    fake_IHC = fake_IHC - fake_IHC.min()
    fake_IHC_norm = fake_IHC / (fake_IHC.max() - fake_IHC.min())

    ssim_scores.append(ssim(fake_IHC, real_IHC).item())
    psnr_scores.append(psnr(fake_IHC, real_IHC).item())

    print(torch.max(real_IHC))
    print(torch.min(real_IHC))
    print(torch.max(fake_IHC_norm))
    print(torch.min(fake_IHC_norm))

result['ssim_mean'].append(np.mean(ssim_scores))
result['ssim_std'].append(np.std(ssim_scores))

result['psnr_mean'].append(np.mean(psnr_scores))
result['psnr_std'].append(np.std(psnr_scores))



tensor(0.9294, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9294, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9294, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9294, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9294, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9294, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)


In [ ]:
def img_batch_plot(IHC_batch, HE_batch, fake_HE_batch, batch_size):
    # prepare figures for plotting 
    fig = plt.figure(figsize=(10, 7))
    rows = batch_size
    columns = 3
    p=0

    for n in range(batch_size):
        p=p+1


        fig.add_subplot(rows, columns, p)
        #IHC_batch[n] = (IHC_batch[n] * 127.5 + 127.5).astype(np.uint8)
        plt.imshow(IHC_batch[n] )
        plt.axis('off')
        plt.title('IHC_img_'+str(n))
        p=p+1

        fig.add_subplot(rows, columns, p)
        #HE_batch[n] = (HE_batch[n] * 127.5 + 127.5).astype(np.uint8)
        plt.imshow(HE_batch[n] )
        plt.axis('off')
        plt.title('HE_img_'+str(n))
        p=p+1

        fig.add_subplot(rows, columns, p)
        #fake_HE_batch[n] = (fake_HE_batch[n] * 127.5 + 127.5).astype(np.uint8)
        plt.imshow(fake_HE_batch[n] )
        plt.axis('off')
        plt.title('predicted_HE_img_'+str(n))

        return fig


def rescale_img_for_plot(IHC_batch,HE_batch,fake_HE_batch):
    # rescale the picture range from [-1..1] to [0..255]
    IHC_batch_256 = (IHC_batch * 127.5 + 127.5).astype(np.uint8)
    HE_batch_256 = (HE_batch * 127.5 + 127.5).astype(np.uint8)
    fake_HE_batch_256 = (fake_HE_batch * 127.5 + 127.5).astype(np.uint8)
    
    return IHC_batch_256,HE_batch_256,fake_HE_batch_256